Modelado de Clasificacion

In [52]:
# Importando librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openpyxl, os
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

#%run "../Modelado/Funciones_Cuota.py"

pd.options.display.float_format = lambda x: f"{x:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
#Desactivar la notación científica
pd.options.display.float_format = '{:.2f}'.format

In [53]:
# Funciones 
#Convertir columnas en fechas
def convertir_columnas_fecha(df, columnas, normalizar=True):
    """
    Convierte múltiples columnas con fechas tipo string (con o sin separadores decimales raros) al formato datetime.
    Opcionalmente normaliza a nivel de fecha (elimina hora y nanosegundos).

    Parámetros:
    - df: DataFrame original.
    - columnas: lista de nombres de columnas a convertir.
    - normalizar: booleano. Si True, aplica .dt.normalize() para dejar solo la fecha.

    Retorna:
    - DataFrame con las columnas especificadas convertidas a datetime (y normalizadas si se desea).
    """
    for col in columnas:
        df[col] = pd.to_datetime(
            df[col].astype(str).str.replace(',', '.', regex=False),
            format='%Y-%m-%d %H:%M:%S.%f',
            errors='coerce'
        )
        if normalizar:
            df[col] = df[col].dt.normalize()
    return df

#Calcular el monto pagado hasta la fecha de vencimiento
def calcular_monto_pagado_hasta_vencimiento(data):
    """
    Calcula el monto acumulado pagado por cada estudiante hasta la fecha de vencimiento de cada cuota.

    Parámetros:
        data (DataFrame): Debe contener las columnas 'CodigoMatricula', 'Fechapago', 'MontoPagado' y 'FechaVencimiento'.

    Retorna:
        DataFrame: Devuelve el mismo DataFrame con una columna nueva 'MontoPagadoAcumFV'.
    """

    # 1. Pagos válidos (con fecha de pago)
    pagos = data[['CodigoMatricula', 'Fechapago', 'MontoPagado']].dropna(subset=['Fechapago']).copy()

    # 2. Cuotas con índice para asignar después
    cuotas = data[['CodigoMatricula', 'FechaVencimiento']].copy()
    cuotas['id_row'] = cuotas.index  # para mapear luego al orden original

    # 3. Merge entre cuotas y pagos
    merged = cuotas.merge(pagos, on='CodigoMatricula', how='left')

    # 4. Filtrar pagos hasta la fecha de vencimiento
    merged = merged[merged['Fechapago'] <= merged['FechaVencimiento']]

    # 5. Agrupar y sumar pagos válidos por fila original
    acumulado = merged.groupby('id_row')['MontoPagado'].sum()

    # 6. Asignar al dataframe original
    data['MontoPagadoAcumFV'] = data.index.to_series().map(acumulado).fillna(0)

    return data

# Numero de cuotas impagas
def calcular_cuotas_impagas(data1):
    """
    Calcula la cantidad de cuotas impagas acumuladas por cada estudiante (CodigoMatricula),
    considerando que una cuota se considera impaga si:
    - No tiene fecha de pago (NaN)
    - O fue pagada después de la fecha de vencimiento de la cuota actual
    
    Parameters:
        data (DataFrame): DataFrame con columnas ['CodigoMatricula', 'NroCuota', 'FechaVencimiento', 'Fechapago']
    
    Returns:
        DataFrame: Mismo DataFrame de entrada con una nueva columna 'CuotasImpagasPrevias'
    """
    
    # Asegurar orden por estudiante y número de cuota
    df = data1.sort_values(by=['CodigoMatricula', 'NroCuota']).reset_index(drop=True)

    # Función interna para contar impagas dentro del grupo
    def contar_impagas(grupo):
        resultado = []
        for i, fila in grupo.iterrows():
            fecha_ref = fila['FechaVencimiento']
            nrocuota_actual = fila['NroCuota']

            # Filtrar cuotas hasta la actual
            cuotas_previas = grupo[grupo['NroCuota'] <= nrocuota_actual]

            # Considerar como impaga si no tiene fecha o pagó después del vencimiento actual
            cuotas_impagas = cuotas_previas[
                (cuotas_previas['Fechapago'].isna()) |
                (cuotas_previas['Fechapago'] > fecha_ref)
            ]

            resultado.append(len(cuotas_impagas))
        grupo['CuotasImpagasPrevias'] = resultado
        return grupo

    # Aplicar por estudiante
    return df.groupby('CodigoMatricula', group_keys=False).apply(contar_impagas)


#### Features Model

In [54]:
info_general=pd.read_csv(r'BD\InfoGeneral.txt', sep='\t', low_memory=False)
# info_general[['CodigoMatricula', 'AreaPrograma','Cargo','NombrePais', 'VersionPrograma', 'Modalidad']].to_csv(r'..\BD\InfoGeneral.txt', sep='\t')

In [55]:
#Importar base de datos
cuotas=pd.read_csv(r'BD\Cuotas.txt', sep='\t',low_memory=False)
info_general=pd.read_csv(r'BD\InfoGeneral.txt', sep='\t', low_memory=False)
actividades=pd.read_csv(r'BD\LlamadasyWhatsap.txt', sep='\t', low_memory=False)
visualizacion=pd.read_csv(r'BD\VisualizacionAonline.txt', sep='\t', low_memory=False)
asistencia=pd.read_csv(r'BD\Asistencia.txt', sep='\t', low_memory=False)
compromisos=pd.read_csv(r'BD\Compromisos.txt',sep='\t',low_memory=False)

In [56]:
# 1. Consolidar base de datos mensual

# Obtener todas las columnas que empiezan con "Fecha" para convertirlas a formato datetime
columFecha = cuotas.columns[cuotas.columns.str.startswith('Fecha')].tolist()
# Convertir las columnas de fechas al formato datetime
cuotas = convertir_columnas_fecha(cuotas, columFecha)

# 2. Filtrar la muestra excluyendo casos atípicos
# Seleccionar columnas de interés para el análisis posterior
colum_elegida_cuota = ['CodigoMatricula', 'NroCuota', 'MontoAdeudado', 'MontoPagado',
    'FechaVencimiento', 'Fechapago', 'FechaMatricula', 'Precio']
# Filtrar el DataFrame:
# - Incluir solo cuotas válidas (no negativas o atípicas)
# - Incluir precios válidos
# - Considerar matrículas entre 2022 y 2024
# - Excluir estudiantes con retiro aprobado o beca
cuotas = cuotas[
    (cuotas['CuotasNegativas2'] == 'Incluir') &
    (cuotas['ExcluirPrecio'] == 'Incluir') &
    (cuotas['FechaMatricula'] >= pd.to_datetime('2022-01-01')) &
    (cuotas['FechaMatricula'] <= pd.to_datetime('2024-12-31')) &
    (~cuotas['EstadoMatricula'].isin(['RETIRO APROBADO', 'BECA']))
][colum_elegida_cuota]

# 3. Construcción de la estructura base de datos para análisis

# Agrupar por matrícula, cuota y precio para consolidar datos de pago y deuda
data = cuotas.groupby(['CodigoMatricula', 'NroCuota', 'Precio']).agg({
    'FechaVencimiento': 'min',      # Mínima fecha de vencimiento por cuota
    'Fechapago': 'max',             # Última fecha de pago registrada
    'MontoPagado': 'sum',           # Total pagado por cuota
    'MontoAdeudado': 'sum'          # Total adeudado por cuota
}).reset_index()

# Asegurar que el número de cuota sea tipo entero
data['NroCuota'] = data['NroCuota'].astype('int64')

# Crear variable binaria que indica si la cuota no fue pagada (1 si no hay fecha de pago)
data['NoPagoTarget'] = data['Fechapago'].isna().astype(int)

# Calcular días de atraso como la diferencia entre fecha de pago y vencimiento
data['AtrasoDias'] = (data['Fechapago'] - data['FechaVencimiento']).dt.days

# Renombrar columna para mayor claridad
data = data.rename(columns={'MontoAdeudado': 'MontoCuota'})

# Ordenar los datos por estudiante y fecha de vencimiento
data = data.sort_values(['CodigoMatricula', 'FechaVencimiento'])
#Calcular el monto total pagado acumulado por estudiante
data['MontoPagadoAcumulado'] = data.groupby('CodigoMatricula')['MontoPagado'].sum()

# 4. Configurar la Fecha de Corte (FechaVencimiento)

# Guardar la fecha de vencimiento real antes de modificarla
data['FechaVencimientoReal'] = data['FechaVencimiento']
# Ajustar la fecha de vencimiento 7 días antes de la fecha real
data['FechaVencimiento'] = data['FechaVencimiento'] - pd.DateOffset(days=7)
# Crear una nueva columna con la fecha de inicio del período de vencimiento (1 mes antes)
data['FechaInicioVencimiento'] = data['FechaVencimiento'] - pd.DateOffset(months=1)

In [57]:
# 5. Merge entre la tabla de cuotas y pagos anteriores

# 5.1 Copiar base de datos original para evitar modificaciones no deseadas
data_copia = data.copy()
# 5.2 Crear subset con columnas relevantes de la tabla principal
data1 = data[['CodigoMatricula', 'NroCuota', 'FechaVencimiento']]
# 5.3 Crear subset con información de pagos anteriores
# Renombrar 'NroCuota' para evitar colisión en el merge
data_copia = data_copia[['CodigoMatricula', 'NroCuota', 'Fechapago', 'AtrasoDias']].rename(
    columns={'NroCuota': 'NroCuota1'})
# 5.4 Realizar merge por 'CodigoMatricula' para cruzar cada cuota con cuotas anteriores
df_merge = data1.merge(data_copia, on='CodigoMatricula', how='left')
df_merge1 = df_merge.copy()

# 6. Cálculo de Atraso Total Acumulado (solo de cuotas anteriores)
# 6.1 Filtrar solo cuotas anteriores a la actual (NroCuota1 < NroCuota) y que el pago haya sido antes del vencimiento actual
df_merge1 = df_merge1[(df_merge1['NroCuota'] > df_merge1['NroCuota1']) &
    (df_merge1['FechaVencimiento'] >= df_merge1['Fechapago'])]

# 6.2 Reemplazar valores negativos de atraso por 0
df_merge1['AtrasoDias'] = np.where(df_merge1['AtrasoDias'] < 0, 0, df_merge1['AtrasoDias'])

# 6.3 Calcular la suma total de días de atraso de cuotas anteriores
data22 = df_merge1.groupby(['CodigoMatricula', 'NroCuota'])['AtrasoDias'].sum().reset_index()
data22 = data22.rename(columns={'AtrasoDias': 'AtrasoSumDias'})

# 7. Cálculo del Promedio de Días de Atraso (sin contar pagos sin atraso)

# 7.1 Filtrar solo los pagos que efectivamente tuvieron atraso (> 0 y no nulos)
data23 = df_merge1[
    (df_merge1['AtrasoDias'] != 0) & (df_merge1['AtrasoDias'].notna())]

# 7.2 Calcular el promedio de días de atraso por cuota actual
data23 = data23.groupby(['CodigoMatricula', 'NroCuota'])['AtrasoDias'].mean().reset_index()
data23 = data23.rename(columns={'AtrasoDias': 'DiasAtrasoPromedio'})

# 8. Añadir las variables calculadas a la base de datos principal

# 8.1 Merge de la suma de atraso total acumulado
data = data.merge(data22, on=['CodigoMatricula', 'NroCuota'], how='left')
# 8.2 Merge del promedio de días de atraso
data = data.merge(data23, on=['CodigoMatricula', 'NroCuota'], how='left')

In [58]:
# 9. Cálculo de Días Desde el Último Pago Anterior

# 9.1 Crear copia del merge anterior para trabajar con seguridad
df_merge2 = df_merge.copy()

# 9.2 Filtrar cuotas anteriores a la actual (NroCuota1 < NroCuota) y donde el pago fue antes o igual al vencimiento de la cuota actual
df_merge2 = df_merge2[
    (df_merge2['NroCuota'] > df_merge2['NroCuota1']) &
    (df_merge2['FechaVencimiento'] >= df_merge2['Fechapago'])]

# 9.3 Para cada cuota actual, obtener la última fecha de pago anterior (más reciente)
df_merge2 = df_merge2.groupby(
    ['CodigoMatricula', 'NroCuota', 'FechaVencimiento']
)['Fechapago'].last().reset_index(name='FechapagoUltimo')

# 9.4 Calcular la cantidad de días transcurridos desde ese último pago hasta el vencimiento de la cuota actual
df_merge2['DiasDesdeUltimoPago'] = (
    df_merge2['FechaVencimiento'] - df_merge2['FechapagoUltimo']).dt.days

# 9.5 Eliminar la columna auxiliar para dejar solo la métrica relevante
df_merge2.drop(columns='FechapagoUltimo', inplace=True)

# 9.6 Incorporar la nueva variable 'DiasDesdeUltimoPago' a la base de datos principal
data = data.merge(df_merge2, on=['CodigoMatricula', 'NroCuota', 'FechaVencimiento'], how='left')


In [59]:
# 10. Limpieza final de datos - Eliminación de estudiantes no válidos

# 10.1 Eliminar estudiantes que no realizaron ningún pago (total pagado = 0 o NaN)// Se tratan de trabajadores de la empresa que reciben este curso gratuitamente

# Calcular el total pagado por cada estudiante
pagoCero = data.groupby('CodigoMatricula')['MontoPagado'].sum()
# Identificar códigos de matrícula con pago cero o sin datos
codigos_sin_pago = pagoCero[(pagoCero == 0) | (pagoCero.isna())].index
# Filtrar fuera del dataset a esos estudiantes
data = data[~data['CodigoMatricula'].isin(codigos_sin_pago)]

# 10.2 Eliminar registros de estudiantes con inconsistencias en el orden de pago

# Estos casos corresponden a alumnos que iniciaron el pago después del vencimiento de su última cuota,
# o cuyo primer pago no corresponde con la primera cuota, lo cual es incoherente ya que toda matrícula válida
# comienza con al menos un pago inicial

# Lista de códigos identificados manualmente como inconsistentes
matriculas_invalidas = ['9909727A23122', '10263740A19230', '10231239A19562', '9508343A21294', '10685735A26612',
    '9522326A21294', '10474261A24199', '10582467A24199', '10238245A18958', '10243608A19563','11062A20631',
    '10138502A20583', '10689709A26723']
# Filtrar estos registros del dataset
data = data[~data['CodigoMatricula'].isin(matriculas_invalidas)]


In [60]:
# 11. Cálculo de días entre cuotas consecutivas
# Se encontró que en el 10% de los registros, la diferencia entre cuotas es mayor a 32 días o menor a 20 días

# Ordenar datos por estudiante y número de cuota
data = data.sort_values(['CodigoMatricula', 'NroCuota'])

# Obtener la fecha de vencimiento real de la cuota anterior para cada estudiante
data['FVCuotaAnterior'] = data.groupby(['CodigoMatricula'])['FechaVencimientoReal'].shift(1)

# Calcular los días entre vencimientos de cuotas consecutivas
data['DiasEntreCuotas'] = (data['FechaVencimientoReal'] - data['FVCuotaAnterior']).dt.days


In [61]:
# 12. Cálculo del total de cuotas anteriores sin atraso

# 12.1 Crear copia de df_merge para manipulación segura
df_merge3 = df_merge.copy()

# 12.2 Reemplazar valores negativos de atraso por cero (no considerar atraso negativo)
df_merge3['AtrasoDias'] = np.where(df_merge3['AtrasoDias'] < 0, 0, df_merge3['AtrasoDias'])

# 12.3 Filtrar solo las cuotas anteriores a la cuota actual y que no tengan atraso (AtrasoDias == 0)
df_merge3 = df_merge3[
    (df_merge3['NroCuota'] > df_merge3['NroCuota1']) &
    (df_merge3['AtrasoDias'] == 0)
]

# 12.4 Contar cuántas cuotas anteriores sin atraso hay para cada estudiante y cuota actual
df_merge3 = df_merge3.groupby(['CodigoMatricula', 'NroCuota']).size().reset_index(name='TotalCuotasSinAtraso')

# 12.5 Incorporar esta nueva variable en la base de datos principal
data = data.merge(df_merge3, on=['CodigoMatricula', 'NroCuota'], how='left')

# 12.6 Reemplazar valores NaN con 0 (para casos sin cuotas sin atraso previas)
data['TotalCuotasSinAtraso'] = data['TotalCuotasSinAtraso'].fillna(0)

In [62]:
# 13. Cálculo de la mayor racha consecutiva de cuotas pagadas sin atraso

# 13.1 Crear copia segura del DataFrame mergeado
df_merge4 = df_merge.copy()

# 13.2 Normalizar valores negativos de atraso a cero (sin atraso)
df_merge4['AtrasoDias'] = np.where(df_merge4['AtrasoDias'] < 0, 0, df_merge4['AtrasoDias'])

# 13.3 Filtrar cuotas anteriores sin atraso para cada cuota actual
df_merge4 = df_merge4[
    (df_merge4['NroCuota'] > df_merge4['NroCuota1']) &
    (df_merge4['AtrasoDias'] == 0)
]

# 13.4 Marcar pagos sin atraso como True (booleano)
df_merge4['PagadaSinAtraso'] = df_merge4['AtrasoDias'] <= 0

# 13.5 Función para calcular la racha máxima consecutiva de pagos sin atraso en una serie ordenada
def max_racha_consecutiva(pagos):
    racha = 0
    max_racha = 0
    for pago_ok in pagos:
        if pago_ok:
            racha += 1
            max_racha = max(max_racha, racha)
        else:
            racha = 0
    return max_racha

# 13.6 Aplicar la función por estudiante y cuota
# Primero ordenar para garantizar secuencia correcta
df_merge4 = df_merge4.sort_values(['CodigoMatricula', 'NroCuota1'])

# Agrupar por alumno y cuota actual, aplicando la función sobre pagos sin atraso
racha_por_cuota = df_merge4.groupby(['CodigoMatricula', 'NroCuota']).agg(
    NroCuotasPagadasConsecutivas = ('PagadaSinAtraso', max_racha_consecutiva)
).reset_index()

# 13.7 Unir resultado al DataFrame principal
data = data.merge(racha_por_cuota, on=['CodigoMatricula', 'NroCuota'], how='left')

# 13.8 Reemplazar valores NaN con 0 para cuotas sin historial previo
data['NroCuotasPagadasConsecutivas'] = data['NroCuotasPagadasConsecutivas'].fillna(0)

# 13.9 Extraer mes de la fecha real de vencimiento de la cuota
data['MesVencimientoCuota'] = data['FechaVencimientoReal'].dt.month


In [63]:
# 14. Preparación para análisis cruzado entre cuotas y pagos

# 14.1 Crear una copia del DataFrame principal para evitar modificaciones directas
data_copia = data.copy()

# 14.2 Seleccionar columnas clave para la tabla base del merge
data1 = data[['CodigoMatricula', 'NroCuota', 'FechaVencimiento']]

# 14.3 Seleccionar columnas relevantes para el merge, renombrando columnas para evitar conflicto de nombres
data_copia = data_copia[['CodigoMatricula', 'NroCuota', 'Fechapago', 'FechaVencimientoReal', 'AtrasoDias']].rename(
    columns={'NroCuota': 'NroCuota1','FechaVencimientoReal': 'FechaVencimientoReal1'})

# 14.4 Realizar merge (left join) entre data1 y data_copia usando CodigoMatricula
df_merge = data1.merge(data_copia, on='CodigoMatricula', how='left')


In [64]:
# 15. Cálculo de los días de morosidad de la cuota anterior

# 15.1 Filtrar filas donde la cuota actual es mayor que la cuota anterior y la fecha de vencimiento de la cuota actual es mayor o igual a la fecha de pago de la cuota anterior
df_merge5 = df_merge[
    (df_merge['NroCuota'] > df_merge['NroCuota1']) &
    (df_merge['FechaVencimiento'] >= df_merge['Fechapago'])]

# 15.2 Filtrar solo los casos donde la cuota anterior es exactamente la cuota anterior inmediata (NroCuota - 1)
df_merge5 = df_merge5[df_merge5['NroCuota'] - 1 == df_merge5['NroCuota1']]

# 15.3 Calcular los días de atraso en la cuota anterior como la diferencia entre la fecha real de vencimiento de la cuota anterior y la fecha de pago de dicha cuota
df_merge5['DiasAtrasoCuotaAnterior'] = (df_merge5['FechaVencimientoReal1'] - df_merge5['Fechapago']).dt.days

# 15.4 Incorporar la variable calculada al DataFrame principal mediante merge
data = data.merge(
    df_merge5[['CodigoMatricula', 'NroCuota', 'DiasAtrasoCuotaAnterior']],
    on=['CodigoMatricula', 'NroCuota'],
    how='left')

In [65]:
# 16. Cálculo del monto pagado en los últimos 30 días antes de la fecha de vencimiento

# 16.1 Seleccionar columnas base para el merge: información de cuotas con fechas relevantes
data1 = data[['CodigoMatricula', 'NroCuota', 'FechaVencimiento', 'FechaInicioVencimiento']]

# 16.2 Seleccionar columnas de pagos, renombrando 'NroCuota' para evitar conflicto en el merge
data_copia = data[['CodigoMatricula', 'NroCuota', 'Fechapago', 'MontoCuota']].rename(columns={'NroCuota': 'NroCuota1'})

# 16.3 Merge left para juntar información de cuotas y pagos por estudiante
df_merge1 = data1.merge(data_copia, on='CodigoMatricula', how='left')

# 16.4 Filtrar pagos que corresponden a cuotas anteriores (NroCuota > NroCuota1), realizados después de la fecha de inicio del período de vencimiento
# y antes de la fecha de vencimiento real de la cuota actual
df_merge7 = df_merge1[
    (df_merge1['NroCuota'] > df_merge1['NroCuota1']) &
    (df_merge1['FechaVencimiento'] > df_merge1['Fechapago']) &
    (df_merge1['Fechapago'] >= df_merge1['FechaInicioVencimiento'])]

# 16.5 Agrupar por estudiante, cuota y fecha de vencimiento y sumar los montos pagados en ese período
df_merge7 = df_merge7.groupby(['CodigoMatricula', 'NroCuota', 'FechaVencimiento'])['MontoCuota'].sum().reset_index(name='PagosUltimos30Dias')

# 16.6 Incorporar la suma de pagos al DataFrame principal
data = data.merge(df_merge7, on=['CodigoMatricula', 'NroCuota', 'FechaVencimiento'], how='left')

In [66]:
# 17. Cálculo de features relacionados a cuotas y montos pagados

# 17.1 Calcular monto pagado acumulado hasta la fecha de vencimiento usando función externa
data = calcular_monto_pagado_hasta_vencimiento(data)

# 17.2 Calcular porcentaje del monto pagado respecto al precio del programa, limitando máximo a 100%
data['%MontoPagadoPrograma'] = (data['MontoPagadoAcumFV'] / data['Precio']).round(3).clip(upper=1) * 100

# 17.3 Calcular número de cuotas impagas usando función externa
data = calcular_cuotas_impagas(data)

# 18. Incorporar información general del estudiante

# 18.1 Seleccionar columnas relevantes de info_general
info_general = info_general[['CodigoMatricula', 'AreaPrograma','Cargo','NombrePais', 'VersionPrograma', 'Modalidad']]

# 18.2 Merge entre data y info_general para añadir atributos demográficos y de programa
data = pd.merge(data, info_general, how='left', on='CodigoMatricula')

# 18.3 Excluir estudiantes con Cargo marcado como 'Nuevo Prueba'
data = data[data.Cargo != 'Nuevo Prueba']

# 19. Procesamiento de llamadas e interacciones

# 19.1 Seleccionar columnas relevantes y convertir fechas
actividades = actividades[['CodigoMatricula', 'FechaActividad', 'LlamadaEfectiva', 'MinutosTotales', 'CCorreosEnviados', 'CWhatsAppEnviados']]
actividades = convertir_columnas_fecha(actividades, ['FechaActividad'])

# 19.2 Definir columnas de conteo y rango para merge
columa_recuento = ['ActividadesTotales', 'ActividadesEfectivas', 'CCorreosEnviados', 'CWhatsAppEnviados', 'MinutosTotales']
evaluar = 'FechaActividad'
colum_merg = ['CodigoMatricula', 'FechaInicioVencimiento', 'FechaVencimiento']

# 19.3 Para cada métrica, hacer merge con data y calcular sumas, conteos o promedios según corresponda
for nueva_columna in columa_recuento:
    if nueva_columna == 'ActividadesTotales':
        df_merge = data[colum_merg].merge(actividades[['CodigoMatricula', evaluar]], on='CodigoMatricula', how='left')
    elif nueva_columna == 'ActividadesEfectivas':
        df_merge = data[colum_merg].merge(
            actividades[['CodigoMatricula', evaluar]][actividades['LlamadaEfectiva'] == 1], on='CodigoMatricula', how='left')
    elif nueva_columna == 'MinutosTotales':
        df_merge = data[colum_merg].merge(
            actividades[['CodigoMatricula', evaluar, nueva_columna]][actividades['LlamadaEfectiva'] == 1], on='CodigoMatricula', how='left')
    elif nueva_columna in ['CCorreosEnviados', 'CWhatsAppEnviados']:
        df_sin_duplicados = actividades[['CodigoMatricula', 'FechaActividad', 'CCorreosEnviados', 'CWhatsAppEnviados']].drop_duplicates()
        df_merge = data[colum_merg].merge(df_sin_duplicados[['CodigoMatricula', evaluar, nueva_columna]], on='CodigoMatricula', how='left')

    # Filtrar actividades que caen dentro del rango de la cuota (entre FechaInicioVencimiento y FechaVencimiento)
    df_filtrado = df_merge.query(f'{evaluar} >= FechaInicioVencimiento and {evaluar} <= FechaVencimiento')

    # Calcular métricas agregadas según tipo de columna
    if nueva_columna in ['ActividadesTotales', 'ActividadesEfectivas']:
        calculo = df_filtrado.groupby(colum_merg).size().reset_index(name=nueva_columna)
    elif nueva_columna in ['CCorreosEnviados', 'CWhatsAppEnviados']:
        calculo = df_filtrado.groupby(colum_merg)[nueva_columna].sum().reset_index(name=nueva_columna)
    elif nueva_columna == 'MinutosTotales':
        calculo = df_filtrado.groupby(colum_merg)[nueva_columna].mean().reset_index(name=nueva_columna)

    # Merge de las métricas calculadas al DataFrame principal
    data = data.merge(calculo, on=colum_merg, how='left')
    data[nueva_columna] = data[nueva_columna].fillna(0).astype(int)

# 20. Calcular tasa de contactabilidad como proporción de actividades efectivas sobre totales
data['TasaContactabilidad'] = np.where(data['ActividadesTotales'] != 0, data['ActividadesEfectivas'] / data['ActividadesTotales'], 0)

# 21. Calcular la primera fecha de vencimiento (primera cuota) por estudiante
data['PrimeraFV'] = data.groupby('CodigoMatricula')['FechaVencimiento'].transform('min')


C:\Users\edosa\AppData\Local\Temp\ipykernel_24084\2389354026.py:98: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby('CodigoMatricula', group_keys=False).apply(contar_impagas)


In [67]:
# 22. Cálculo de la variable: ContactoEfectivoUltimos7D
# Esta variable indica si hubo una llamada efectiva (respondida) en los 7 días previos a la fecha de vencimiento de la cuota.

# 22.1 Filtrar solo las actividades con llamadas efectivas
actividades_copy = actividades[actividades.LlamadaEfectiva == 1][['CodigoMatricula', 'FechaActividad']]

# 22.2 Hacer merge con las cuotas para asociar las actividades con cada cuota por estudiante
actividades_copy = data[['CodigoMatricula', 'NroCuota', 'FechaVencimiento']].merge(
    actividades_copy,
    on='CodigoMatricula',
    how='left'
)

# 22.3 Filtrar actividades que ocurrieron antes o en la fecha de vencimiento de la cuota
actividades_copy = actividades_copy[actividades_copy.FechaVencimiento >= actividades_copy.FechaActividad]

# 22.4 Obtener la última fecha de llamada efectiva antes de la cuota, por estudiante y cuota
actividades_copy = actividades_copy.groupby(['CodigoMatricula', 'NroCuota', 'FechaVencimiento'])['FechaActividad']\
    .last().reset_index(name='UltimaActividadEfectiva')

# 22.5 Calcular la cantidad de días entre la fecha de vencimiento y la última actividad efectiva
actividades_copy['Dias'] = (actividades_copy['FechaVencimiento'] - actividades_copy['UltimaActividadEfectiva']).dt.days

# 22.6 Crear variable binaria: 1 si hubo contacto efectivo en los últimos 7 días, 0 si no
actividades_copy['ContactoEfectivoUltimos7D'] = (actividades_copy['Dias'] <= 7).astype(int)

# 22.7 Merge final con la base principal
data = data.merge(
    actividades_copy[['CodigoMatricula', 'NroCuota', 'FechaVencimiento', 'ContactoEfectivoUltimos7D']],
    on=['CodigoMatricula', 'NroCuota', 'FechaVencimiento'],
    how='left')

# 22.8 Rellenar con 0 los valores faltantes (casos sin actividad efectiva registrada)
data['ContactoEfectivoUltimos7D'] = data['ContactoEfectivoUltimos7D'].fillna(0)

In [68]:
# 23. Incorporación de datos de visualización de contenidos (videos)

# 23.1 Definir columnas necesarias para el análisis de visualización
colum_visualizacion = [
    'CodigoMatricula', 'OrdenCapitulo', 'Capitulo',
    'NumeroSeccion', 'OrdenSeccion','DuracionVideo', 'EstadoVideo',
    'FechaFinVisualizacion', 'Duración Final', 'FechaFinPrograma', 'FechaMatricula'
]

# 23.2 Filtrar registros eliminando estudiantes con estado atípico
visualizacion = visualizacion[visualizacion.AtipicoEstadoAlumno2 != 'No Considerar'][colum_visualizacion]

# 23.3 Convertir columnas de fechas al tipo datetime
visualizacion = convertir_columnas_fecha(visualizacion, ['FechaFinVisualizacion', 'FechaFinPrograma', 'FechaMatricula'])

# 23.4 Asegurar que 'EstadoVideo' esté en tipo float para permitir operaciones agregadas
visualizacion['EstadoVideo'] = visualizacion['EstadoVideo'].astype('float')

# 23.5 Agrupar para obtener el primer registro de visualización por sección y el estado máximo de avance
visualizacion = visualizacion.groupby([
    'CodigoMatricula', 'OrdenCapitulo', 'Capitulo',
    'NumeroSeccion', 'OrdenSeccion', 'DuracionVideo',
    'FechaFinPrograma', 'FechaMatricula' \
]).agg({'FechaFinVisualizacion': 'min',
    'EstadoVideo': 'max'
}).reset_index()

# 23.6 Calcular la duración total del programa para cada estudiante
visualizacion_total = visualizacion.groupby('CodigoMatricula')['DuracionVideo'].sum().reset_index(name='DuracionTotal')

# 23.7 Eliminar registros sin visualización registrada
visualizacion = visualizacion[~visualizacion['FechaFinVisualizacion'].isna()]

# 23.8 Filtrar visualizaciones únicamente para estudiantes presentes en la tabla principal `data`
codig_visual = np.intersect1d(visualizacion.CodigoMatricula.unique(), data.CodigoMatricula.unique())
visualizacion = visualizacion[visualizacion.CodigoMatricula.isin(codig_visual)][[
    'CodigoMatricula', 'FechaFinVisualizacion', 'DuracionVideo'
]]

# 23.9 Hacer merge con las cuotas, para evaluar qué videos fueron vistos antes de la fecha de vencimiento
df_merge = data[data.CodigoMatricula.isin(codig_visual)][
    ['CodigoMatricula', 'NroCuota', 'FechaVencimiento']
].merge(visualizacion, on='CodigoMatricula', how='left')

# 23.10 Filtrar visualizaciones que ocurrieron antes o en la fecha de vencimiento de la cuota correspondiente
df_merge = df_merge[df_merge['FechaFinVisualizacion'] <= df_merge['FechaVencimiento']]

# 23.11 Calcular el total de minutos visualizados antes de la fecha de vencimiento por estudiante y cuota
df_sum = df_merge.groupby(['CodigoMatricula', 'NroCuota', 'FechaVencimiento'])['DuracionVideo']\
    .sum().reset_index(name='DuracionAvanzado')

# 23.12 Incorporar esta duración avanzada al dataset principal
data = data.merge(df_sum, on=['CodigoMatricula', 'FechaVencimiento', 'NroCuota'], how='left')

# 23.13 Incorporar la duración total del programa al dataset principal
data = data.merge(visualizacion_total, on='CodigoMatricula', how='left')

In [69]:
# 24. Cálculo de la variable: DiasUltVisualizacion
# Esta variable representa los días que han pasado desde la última visualización de contenido hasta la fecha de vencimiento de la cuota.

# 24.1 Obtener la última fecha de visualización registrada antes del vencimiento, por estudiante y cuota
df_merge = df_merge.groupby(['CodigoMatricula', 'NroCuota', 'FechaVencimiento'])['FechaFinVisualizacion']\
    .last().reset_index(name='UltimaFechaVisualizacion')

# 24.2 Calcular la diferencia en días entre la fecha de vencimiento y la última visualización
df_merge['DiasUltVisualizacion'] = (df_merge['FechaVencimiento'] - df_merge['UltimaFechaVisualizacion']).dt.days

# 24.3 Hacer merge con la base principal para incorporar DiasUltVisualizacion
data = data.merge(
    df_merge[['CodigoMatricula', 'NroCuota', 'FechaVencimiento', 'DiasUltVisualizacion']],
    on=['CodigoMatricula', 'NroCuota', 'FechaVencimiento'],
    how='left')

In [70]:
# 25. Incorporación de datos de asistencia y cálculo del porcentaje de avance en el programa

# 25.1 Seleccionar columnas relevantes del dataset de asistencia y eliminar duplicados
colum_asistencia = [
    'CodigoMatricula', 'CentroCostoPadre', 'CentroCosto', 'Curso',
    'FechaSesion', 'EstadoAsistencia', 'FechaInicioPrograma', 'FechaTerminoPrograma']
asistencia = asistencia[colum_asistencia].drop_duplicates()

# 25.2 Convertir fechas al formato datetime
asistencia = convertir_columnas_fecha(asistencia, ['FechaSesion', 'FechaInicioPrograma', 'FechaTerminoPrograma'])

# 25.3 Filtrar estudiantes que están presentes tanto en asistencia como en la tabla principal
codig_asistencia = np.intersect1d(asistencia.CodigoMatricula.unique(), data.CodigoMatricula.unique())
asistencia = asistencia[asistencia['CodigoMatricula'].isin(codig_asistencia)]

# 25.4 Asociar cada registro de asistencia con su respectiva cuota, solo si la sesión fue antes del vencimiento
df_merge = asistencia[['CodigoMatricula', 'FechaSesion', 'EstadoAsistencia']].set_index('CodigoMatricula').join(
    data[['CodigoMatricula', 'FechaVencimiento']][data.CodigoMatricula.isin(codig_asistencia)].set_index('CodigoMatricula')
)
df_merge = df_merge[df_merge.FechaSesion <= df_merge.FechaVencimiento]

# 25.5 Calcular la cantidad de asistencias por estudiante y cuota
data = data.merge(
    df_merge.groupby(['CodigoMatricula', 'FechaVencimiento'])['EstadoAsistencia'].sum().reset_index(name='Asistencias'),
    on=['CodigoMatricula', 'FechaVencimiento'],
    how='left'
)

# 25.6 Completar valores faltantes de asistencia y duración avanzada
data['Asistencias'] = data['Asistencias'].fillna(0)
data['DuracionAvanzado'] = data['DuracionAvanzado'].fillna(0)

# 25.7 Agregar el total de sesiones del estudiante
data = data.merge(
    df_merge.groupby(['CodigoMatricula'])['FechaSesion'].size().reset_index(name='SesionesTotales'),
    on='CodigoMatricula',
    how='left')

# 25.8 Calcular el porcentaje de avance del estudiante en el programa
# - Para modalidad asincrónica: avance basado en minutos de video vistos.
# - Para modalidad sincrónica: avance basado en número de asistencias a clases.
data['PctAvanzado'] = np.where(
    data['Modalidad'] == 'Online Asincronica',
    data['DuracionAvanzado'] / data['DuracionTotal'],
    data['Asistencias'] / data['SesionesTotales'])

# 25.9 Calcular el número total de cuotas del estudiante
data.loc[:, 'TotalCuotas'] = data.groupby(['CodigoMatricula'])['NroCuota'].transform('max')


In [71]:
# 26. Cálculo de la variable: DiasUltAsistencia
# Esta variable indica cuántos días antes de la fecha de vencimiento fue la última asistencia registrada del estudiante.

# 26.1 Filtrar solo registros donde el estudiante asistió efectivamente a la clase
df_merge = df_merge[df_merge.EstadoAsistencia == 1]

# 26.2 Agrupar por estudiante y cuota para obtener la fecha de la última asistencia antes del vencimiento
df_merge = df_merge.reset_index().groupby(['CodigoMatricula', 'FechaVencimiento'])['FechaSesion']\
    .last().reset_index(name='UltFechaAsistencia')

# 26.3 Calcular los días entre la fecha de vencimiento de la cuota y la última asistencia
df_merge['DiasUltAsistencia'] = (df_merge['FechaVencimiento'] - df_merge['UltFechaAsistencia']).dt.days

# 26.4 Integrar esta nueva variable en la base de datos principal
data = data.merge(
    df_merge[['CodigoMatricula', 'FechaVencimiento', 'DiasUltAsistencia']],
    on=['CodigoMatricula', 'FechaVencimiento'],
    how='left')

In [72]:
# 27. Cálculo de la variable: DiasUltimoAvance
# Esta variable representa cuántos días han pasado desde la última actividad de avance del estudiante,
# considerando el tipo de modalidad del programa.

# - Para modalidad 'Online Asincronica': se usa la fecha de última visualización de contenido.
# - Para modalidad sincrónica (no asincrónica): se usa la fecha de la última asistencia registrada.

data['DiasUltimoAvance'] = np.where(
    data['Modalidad'] == 'Online Asincronica',
    data['DiasUltVisualizacion'],
    data['DiasUltAsistencia'])

In [73]:
# 28. Cálculo de la variable: TotalCompromisosCuotaAnterior
# Esta variable representa la cantidad de compromisos de pago registrados para la cuota inmediatamente anterior a la actual.

# 28.1 Crear columna que represente la cuota anterior (para poder hacer el merge correctamente)
compromisos['NroCuotaAnterior'] = compromisos['NroCuota'] + 1  # Se suma 1 para que coincida con la cuota actual en 'data'

# 28.2 Agrupar compromisos por estudiante y cuota anterior, contando cuántos compromisos existen
resum_compromisos = compromisos.groupby(['CodigoMatricula', 'NroCuotaAnterior'])['FechaCompromiso']\
    .size().reset_index(name='TotalCompromisosCuotaAnterior')

# 28.3 Crear columna auxiliar en el dataset principal para hacer el merge
data['NroCuotaAnterior'] = data['NroCuota']

# 28.4 Unir la información de compromisos al dataset principal
data = data.merge(resum_compromisos, on=['CodigoMatricula', 'NroCuotaAnterior'], how='left')

# 28.5 Rellenar con 0 los valores nulos (casos sin compromisos)
data['TotalCompromisosCuotaAnterior'] = data['TotalCompromisosCuotaAnterior'].fillna(0)

# 28.6 Agregar la moneda de la cuota para cada estudiante
data = data.merge(
    compromisos[['CodigoMatricula', 'MonedaCuota']].drop_duplicates(),
    on='CodigoMatricula',
    how='left'
)

In [74]:
# 29. Filtrado de estudiantes que no pagaron la primera cuota
# Se identifican los estudiantes cuya primera cuota (NroCuota = 1) no tiene fecha de pago (AtrasoDias = NaN)
# y se excluyen del análisis, ya que representan casos atípicos (nunca realizaron un primer pago).
codigoM_Cuota = data[(data['NroCuota'] == 1) & (data.AtrasoDias.isna())]['CodigoMatricula'].unique()
data = data[~(data.CodigoMatricula.isin(codigoM_Cuota))]

# 30. Transformación de la variable Modalidad
# Se convierte en variable binaria, donde 'Online Sincronica' se codifica como 1 y el resto como 0.
data['Modalidad'] = (data['Modalidad'] == 'Online Sincronica').astype(int)

# 31. Guardar una copia del dataset antes de aplicar limpieza por PctAvanzado
# Esto puede ser útil para análisis o validaciones posteriores.
data_antes_limpieza = data.copy()

# 32. Filtrado de registros sin información del porcentaje de avance
# Se excluyen registros donde 'PctAvanzado' es nulo, ya que no se podría calcular la progresión del estudiante.
data = data[~(data.PctAvanzado.isna())]

# 33. Cálculo de variables finales
# 33.1 Ordenar los datos por estudiante y cuota
data = data.sort_values(['CodigoMatricula', 'NroCuota'])

# 33.2 Calcular número de cuotas restantes
data['NumCuotasRestantes'] = data['TotalCuotas'] - data['NroCuota']

# 33.3 Calcular el desfase entre el avance académico y el pago
# La fórmula compara el % pagado con el % avanzado. El +100 compensa la escala relativa y normaliza la métrica.
data['DesfaseEntreAvanceYPago'] = (data['%MontoPagadoPrograma'] - data['PctAvanzado'] * 100) + 100


In [75]:
Bd_Final=data[['CodigoMatricula', 'NroCuota','DiasDesdeUltimoPago','FechaVencimiento','Fechapago','FechaVencimientoReal',
        '%MontoPagadoPrograma', 'CuotasImpagasPrevias',
       'AreaPrograma','NombrePais','VersionPrograma', 'Modalidad', 
       'ActividadesEfectivas', 'CCorreosEnviados', 'CWhatsAppEnviados','MinutosTotales','TotalCompromisosCuotaAnterior',
       'TasaContactabilidad','ActividadesTotales','PctAvanzado', 'MonedaCuota','DesfaseEntreAvanceYPago',
        'MesVencimientoCuota','DiasAtrasoPromedio','NroCuotasPagadasConsecutivas','NumCuotasRestantes',
        'DiasEntreCuotas','ContactoEfectivoUltimos7D','TotalCuotasSinAtraso','DiasAtrasoCuotaAnterior','AtrasoDias','DiasUltimoAvance',
        'PagosUltimos30Dias']]

In [76]:
# 34. Guardar el DataFrame final en un archivo pickle para uso posterior
Bd_Final.to_pickle(r'Resultado/DataCumplimientoCuota_7days_CreandoVar1.pickle')